# Baselines and their results

In [ ]:
from gcn import *
from gcn_perturbed import *
import torch
import numpy

## Random baseline

Steps of the random baseline (according to the paper):
1. Randomly initialize P_hat in [-1,1]
2. Apply sigmoid and threshold
3. Do this for K times, keep minimal CF

In [ ]:
def random_baseline(subgraph, node):
    pass

In [ ]:
for number in k:
    pert_matrix = random_baseline(sub_adj, node_index)
    new_matrix = torch.mul(sub_adj, pert_matrix)
    sparse_adj_test = get_sparse_adjacency_normalized(features_perturbed.shape[0], adjacency_matrix)
    
    if old_prediction != model()

## 1-Hop baseline and remove 1-Hop baseline

Steps of the 1-Hop baseline (according to the paper):
1. Get ego-graph of node
2. Remove every edge not in the ego graph

Steps of the Remove 1-Hop baseline (according to the paper):
1. Get ego-graph of node
2. Remove every edge in the ego graph

In [ ]:
def one_hop_baselines(subgraph, node):
    # return both the perturbation matrix for the one-hop and remove one-hop baselines IF it is a explanation
    pass

## GNNExplainer baseline

Steps of the GNNExplainer baseline (according to the paper):
1. Identify the S most relevent edges in prediction.
2. Remove these edges
3. S in {1,2,3,4,5,GT}

(GT is 6 for Syn1, 6 for Syn4 and 12 for Syn5, according to the paper)

In [ ]:
def gnnexplainer_baseline(nr_edges, subgraph, node, model):
    pass